IMPORTANDO BIBLIOTECAS QUE IRÃO SER UTILIZADAS NO PROJETO


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read, write
from numpy.fft import fft, ifft, fftfreq, fftshift
from IPython.display import Audio as play

FUNÇÃO AUXILIAR PARA PLOTAGEM DE UM VETOR NO DOMÍNIO DA FREQUÊNCIA EM dB

In [3]:
def spectrum(x_freq):
# 
# x_freq: vetor no domínio da frequência, complexo
#
    x_magnitude = np.abs(x)
    # Normalização para o valor máximo ser 0dB
    x_magnitude /= np.max(x_magnitude)
    return 20*np.log10(x_magnitude)

PRIMEIRO PASSO: Ler o arquivo .wav para obter o sinal x[n] em forma de vetor.